# 01 – SQL Ingestion and Cleaning (SQLite)

This notebook simulates the SQL stage of a production analytics pipeline.

**Goals:**
- Ingest the raw mobile reviews CSV into a local **SQLite** database.
- Create a `reviews_raw` table that mirrors the source data.
- Use **SQL transformations** to:
  - Standardize key columns (brand, model, text, rating, date).
  - Filter out invalid or empty records.
  - Materialize a cleaned table `reviews_clean`.
- Optionally export `reviews_clean` back to CSV for downstream Python/NLP steps.

This SQL layer demonstrates the “Python + SQL pipeline” aspect of the
**Proactive Device Quality Insight Pipeline** project.


### 1. Imports & Paths

In [1]:
import pandas as pd
import sqlite3
from pathlib import Path

# Path to the raw CSV (adjust if your path is slightly different)
csv_path = Path("/Volumes/Personal Drive/GitHub/Proactive-Device-Quality-Signal-Detection/Dataset/Mobile Reviews Sentiment.csv")

# Path to the SQLite database we will create
db_path = Path("/Volumes/Personal Drive/GitHub/Proactive-Device-Quality-Signal-Detection/Dataset/device_quality.db")

csv_path, csv_path.exists(), db_path


(PosixPath('/Volumes/Personal Drive/GitHub/Proactive-Device-Quality-Signal-Detection/Dataset/Mobile Reviews Sentiment.csv'),
 True,
 PosixPath('/Volumes/Personal Drive/GitHub/Proactive-Device-Quality-Signal-Detection/Dataset/device_quality.db'))

#### 1.1 Preview of Raw CSV

In [2]:
# Load a small preview of the CSV
df_raw = pd.read_csv(csv_path)

df_raw.head()


,review_id,customer_name,age,brand,model,price_usd,price_local,currency,exchange_rate_to_usd,rating,...,verified_purchase,battery_life_rating,camera_rating,performance_rating,design_rating,display_rating,review_length,word_count,helpful_votes,source
0,1,Aryan Maharaj,45,Realme,Realme 12 Pro,337.31,₹27996.73,INR,83.00,2,...,True,1,1,3,2,1,46,7,1,Amazon
1,2,Davi Miguel Sousa,18,Realme,Realme 12 Pro,307.78,R$1754.35,BRL,5.70,4,...,True,3,2,4,3,2,74,12,5,Flipkart
2,3,Pahal Balay,27,Google,Pixel 6,864.53,₹71755.99,INR,83.00,4,...,True,3,5,3,2,4,55,11,8,AliExpress
3,4,David Guzman,19,Xiaomi,Redmi Note 13,660.94,د.إ2425.65,AED,3.67,3,...,False,1,3,2,1,2,66,11,3,Amazon
4,5,Yago Leão,38,Motorola,Edge 50,792.13,R$4515.14,BRL,5.70,3,...,True,3,3,2,2,1,73,12,0,BestBuy


In [3]:
df_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   review_id             50000 non-null  int64  
 1   customer_name         50000 non-null  object 
 2   age                   50000 non-null  int64  
 3   brand                 50000 non-null  object 
 4   model                 50000 non-null  object 
 5   price_usd             50000 non-null  float64
 6   price_local           50000 non-null  object 
 7   currency              50000 non-null  object 
 8   exchange_rate_to_usd  50000 non-null  float64
 9   rating                50000 non-null  int64  
 10  review_text           50000 non-null  object 
 11  sentiment             50000 non-null  object 
 12  country               50000 non-null  object 
 13  language              50000 non-null  object 
 14  review_date           50000 non-null  object 
 15  verified_purchase  

### 2. Create SQLite Database and Ingest Raw Data

We now create (or overwrite) a local SQLite database file and write the
full raw CSV into a table called `reviews_raw`.

This simulates an ingestion step where raw data lands in a SQL warehouse.


In [4]:
# Create (or connect to) the SQLite database
conn = sqlite3.connect(db_path)

# Ingest the raw dataframe into a SQL table named "reviews_raw"
df_raw.to_sql("reviews_raw", conn, if_exists="replace", index=False)

# Confirm row count
row_count = conn.execute("SELECT COUNT(*) FROM reviews_raw;").fetchone()[0]
row_count


50000

### 3. Inspect `reviews_raw` via SQL

We run a couple of simple SQL queries to confirm:

- Column names and sample rows.
- Basic distributions (e.g., min/max dates, rating ranges).

This also demonstrates using SQL directly rather than relying only on pandas.


In [5]:
# Peek at a few rows
sample_rows = conn.execute("""
    SELECT review_id, brand, model, rating, review_date, review_text
    FROM reviews_raw
    LIMIT 5;
""").fetchall()

sample_rows


[(1,
  'Realme',
  'Realme 12 Pro',
  2,
  '2023-11-06',
  'Not worth the money spent. Wouldn’t recommend.'),
 (2,
  'Realme',
  'Realme 12 Pro',
  4,
  '2023-03-30',
  'Absolutely love this phone! The camera is next level. Absolutely worth it!'),
 (3,
  'Google',
  'Pixel 6',
  4,
  '2022-12-07',
  'Loving the clean UI and fast updates. Loving it so far!'),
 (4,
  'Xiaomi',
  'Redmi Note 13',
  3,
  '2025-03-11',
  'Build quality feels solid and durable. No regrets buying this one.'),
 (5,
  'Motorola',
  'Edge 50',
  3,
  '2023-09-29',
  'Not bad for daily use but could be optimized. Average experience overall.')]

In [6]:
# check rating stats

rating_stats = conn.execute("""
    SELECT 
        MIN(rating) AS min_rating,
        MAX(rating) AS max_rating,
        AVG(rating) AS avg_rating
    FROM reviews_raw;
""").fetchone()

rating_stats


(1, 5, 3.12312)

### 4. SQL-Based Cleaning: `reviews_clean`

We now create a cleaned table `reviews_clean` using pure SQL.

Cleaning rules:
- Keep only key columns:
  - `review_id`
  - `brand` → `device_brand`
  - `model` → `device_model`
  - `rating`
  - `review_text`
  - `review_date`
- Drop rows where:
  - `review_text` is NULL or empty after trimming.
  - `review_date` is NULL.
- Optionally filter rating to a reasonable range (1–5).

This step represents the **SQL transformation layer** in the pipeline.


In [7]:
# Drop existing cleaned table if it exists
conn.execute("DROP TABLE IF EXISTS reviews_clean;")

# Create the cleaned table using SQL
create_clean_table_sql = """
CREATE TABLE reviews_clean AS
SELECT
    review_id,
    TRIM(brand) AS device_brand,
    TRIM(model) AS device_model,
    rating,
    TRIM(review_text) AS review_text,
    review_date
FROM reviews_raw
WHERE review_text IS NOT NULL
  AND TRIM(review_text) <> ''
  AND review_date IS NOT NULL
  AND rating IS NOT NULL
  AND rating BETWEEN 1 AND 5;
"""

conn.execute(create_clean_table_sql)
conn.commit()

# Confirm row count in cleaned table
clean_count = conn.execute("SELECT COUNT(*) FROM reviews_clean;").fetchone()[0]
clean_count


50000

### 5. Validate `reviews_clean`

We confirm that:
- The cleaned table contains the expected columns.
- Obvious junk (empty text, missing dates) has been removed.
- A few sample rows look reasonable.


In [9]:
sample_clean = conn.execute("""
    SELECT device_brand, device_model, rating, review_date, review_text
    FROM reviews_clean
    LIMIT 5;
""").fetchall()

sample_clean


[('Realme',
  'Realme 12 Pro',
  2,
  '2023-11-06',
  'Not worth the money spent. Wouldn’t recommend.'),
 ('Realme',
  'Realme 12 Pro',
  4,
  '2023-03-30',
  'Absolutely love this phone! The camera is next level. Absolutely worth it!'),
 ('Google',
  'Pixel 6',
  4,
  '2022-12-07',
  'Loving the clean UI and fast updates. Loving it so far!'),
 ('Xiaomi',
  'Redmi Note 13',
  3,
  '2025-03-11',
  'Build quality feels solid and durable. No regrets buying this one.'),
 ('Motorola',
  'Edge 50',
  3,
  '2023-09-29',
  'Not bad for daily use but could be optimized. Average experience overall.')]

In [12]:
#In pandas

df_clean_sql = pd.read_sql("SELECT * FROM reviews_clean LIMIT 10;", conn)
df_clean_sql


,review_id,device_brand,device_model,rating,review_text,review_date
0,1,Realme,Realme 12 Pro,2,Not worth the money spent. Wouldn’t recommend.,2023-11-06
1,2,Realme,Realme 12 Pro,4,Absolutely love this phone! The camera is next...,2023-03-30
2,3,Google,Pixel 6,4,Loving the clean UI and fast updates. Loving i...,2022-12-07
3,4,Xiaomi,Redmi Note 13,3,Build quality feels solid and durable. No regr...,2025-03-11
4,5,Motorola,Edge 50,3,Not bad for daily use but could be optimized. ...,2023-09-29
5,6,Apple,iPhone 14,5,Battery easily lasts a day with heavy use. No ...,2023-06-27
6,7,OnePlus,OnePlus 12,3,Loving the clean UI and fast updates. Absolute...,2024-03-27
7,8,Apple,iPhone SE,2,"Phone hangs often, regret buying it. Wouldn’t ...",2024-04-08
8,9,Realme,Realme 12 Pro,4,Battery easily lasts a day with heavy use. Lov...,2024-10-04
9,10,OnePlus,OnePlus Nord 3,3,Smooth performance even after months of use. N...,2025-02-25


### 6. Export `reviews_clean` to CSV

To keep the downstream Python/NLP steps simple, we can export the cleaned
SQL table `reviews_clean` to a separate CSV.

Later notebooks can either:
- Read from this CSV, or
- Read directly from the SQLite database.

Both are valid; the important part is that the **cleaning logic lives in SQL**.


In [13]:
clean_csv_path = Path("/Volumes/Personal Drive/GitHub/Proactive-Device-Quality-Signal-Detection/Dataset/mobile_reviews_clean_sql.csv")

df_clean_full = pd.read_sql("SELECT * FROM reviews_clean;", conn)
df_clean_full.to_csv(clean_csv_path, index=False)

clean_csv_path


PosixPath('/Volumes/Personal Drive/GitHub/Proactive-Device-Quality-Signal-Detection/Dataset/mobile_reviews_clean_sql.csv')

### 7. Close Database Connection

We close the SQLite connection cleanly. The database file
`device_quality.db` now contains:

- `reviews_raw` — direct ingestion of the raw CSV
- `reviews_clean` — SQL-cleaned and standardized reviews

This completes the SQL ingestion and cleaning stage of the pipeline.


In [14]:
conn.close()